# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/electricity/4_electricity_batch_inference.ipynb)

## 🗒️ This notebook is divided into the following sections:

1. Load batch data.
2. Predict using model from Model Registry.

## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import joblib
import inspect 
import datetime
import pandas as pd
import tensorflow as tf

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27;"> ⚙️ Feature View Retrieval</span>


In [ ]:
feature_view = fs.get_feature_view(
    name='electricity_feature_view',
    version=1,
)

## <span style="color:#ff5f27;">🗄 Model Registry</span>


In [ ]:
mr = project.get_model_registry()

## <span style="color:#ff5f27;"> 📮 Retrieving model from Model Registry </span>

In [ ]:
model = mr.get_model("electricity_price_prediction_model", version=1)
model_dir = model.download()
loaded_model = tf.saved_model.load(model_dir)
serving_function = loaded_model.signatures["serving_default"]

## <span style="color:#ff5f27;"> ✨ Load Batch Data</span>

In [ ]:
import datetime
start_date = datetime.datetime.now() - datetime.timedelta(days=5)
start_time = int(start_date.timestamp()) * 1000

In [ ]:
feature_view = fs.get_feature_view("electricity_feature_view", 1)
feature_view.init_batch_scoring(training_dataset_version=1)
df = feature_view.get_batch_data(start_time=start_time)
df.sort_values(["timestamp"], inplace=True)
df = df.drop(["day", "timestamp"], axis=1)
df

## <span style="color:#ff5f27;"> 🤖 Making the predictions </span>

In [ ]:
x = serving_function(tf.constant(df.values.reshape(-1, df.shape[0], df.shape[1]), tf.float32))
prediction = x[list(x.keys())[0]].numpy() #.flatten().tolist()
prediction

In [ ]:
price_se1_pred = []
price_se2_pred = []
price_se3_pred = []
price_se4_pred = []

for batch_n in range(len(prediction)):
    batch = prediction[batch_n]
    for window_n in range(df.shape[0]):
        price_se1_pred.append(batch[window_n][0])
        price_se2_pred.append(batch[window_n][1])
        price_se3_pred.append(batch[window_n][2])
        price_se4_pred.append(batch[window_n][3])

In [ ]:
feature_view.init_serving(1)

td_transformation_functions = feature_view._batch_scoring_server._transformation_functions

In [ ]:
# decoding our features

preds = pd.DataFrame(data={'price_se1': price_se1_pred,
                           'price_se2': price_se2_pred,
                           'price_se3': price_se3_pred,
                           'price_se4': price_se4_pred,})

res = {}
for feature_name in td_transformation_functions:
    if feature_name in ["price_se1", "price_se2", "price_se3", "price_se4"]:
        td_transformation_function = td_transformation_functions[feature_name]
        sig, foobar_locals = inspect.signature(td_transformation_function.transformation_fn), locals()
        param_dict = dict([(param.name, param.default) for param in sig.parameters.values() if param.default != inspect._empty])
        if td_transformation_function.name == "min_max_scaler":
            preds[feature_name] = preds[feature_name].map(
                lambda x: x * (param_dict["max_value"] - param_dict["min_value"]) + param_dict["min_value"]
                )
            
preds = preds.apply(lambda x: -x)
preds

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

%config InlineBackend.figure_format='retina'
%matplotlib inline

plt.plot(preds["price_se1"],color='red', label='SE1 price prediction')
plt.plot(preds["price_se2"], color='blue', label='SE2 price prediction')
plt.plot(preds["price_se3"], color='green', label='SE3 price prediction')
plt.plot(preds["price_se4"], color='black', label='SE4 price prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend(loc='upper left')

fig = plt.gcf()
fig.set_size_inches(15, 5)

---

### <span style="color:#ff5f27;">🥳 <b> Next Steps  </b> </span>
Congratulations you've now completed the Electricity price tutorial for Managed Hopsworks.

Check out our other tutorials on ➡ https://github.com/logicalclocks/hopsworks-tutorials

Or documentation at ➡ https://docs.hopsworks.ai